# Demo on DLPFC tissue
We use of section 151509 of the human dorsolateral prefrontal cortex (DLPFC) tissue as an example to show how to use srstGCN to obtain super-resolution gene expression profiles by weakly supervised GCN framework. srstGCN enhances gene expression to single-cell resolution and can also predict gene expression both outside the spots and in external tissue sections at the single-cell level. The 10X Visium data of section 151509 can be downloaded from [https://zenodo.org/records/12800375](https://zenodo.org/records/12800375)

# Data preparation

In [1]:
import align_image_pos
import os
prefix =f'{os.getcwd()}\\data\\'
align_image_pos.main(prefix)

C:\Users\DELL\anaconda3\envs\pytorch\Lib\site-packages\PIL\Image.py:3157: DecompressionBombWarning: Image size (177742224 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


# Mutimodel feature map extraction

In [2]:
from histological_feature_map import main as his_ex
from pos_feature_map import main as pos_ex
from RGB_feature_map import main as rgb_ex
import pickle
login = 'login'  # Please change to your applied UNI login
img_emb = his_ex(prefix, login)
rgb_emb = rgb_ex(prefix)
pos_emb = pos_ex(prefix, img_emb)
embs = dict(his=img_emb, rgb=rgb_emb, pos=pos_emb)
with open(prefix + 'Multimodal_feature_map.pickle', 'wb') as file:
    pickle.dump(embs, file)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\DELL\.cache\huggingface\token
Login successful
The size of the histological image is:(19264, 19264, 3)


C:\Users\DELL\anaconda3\envs\pytorch\Lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  x = F.scaled_dot_product_attention(


# Select highly variable genes and generate mask image

In [3]:
import get_mask
from utils import get_HVG_genes
get_mask.main(prefix)
get_HVG_genes(prefix, n_top=1000)

Clustering pixels using km...
(1373584, 1029)
n_clusters: 2


# Inferring sing-cell resolution gene expression after the train process

In [4]:
from impute import main as train_model
# The default value of n_states is set to `5`
train_model(prefix, epoch=500, device='cuda', n_states=1, load_saved=False)

x: (1204, 1204, 1029) , y: (4789, 1000)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | GCN_module    | Sequential | 788 K 
1 | output_module | Linear     | 512 K 
---------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.204     Total estimated model params size (MB)
C:\Users\DELL\anaconda3\envs\pytorch\Lib\site-packages\pytorch_lightning\trainer\connectors\data_co

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=500` reached.


218 sec
Model saved to E:\scstGCN-main\data\states/00/model.pt
History saved to E:\scstGCN-main\data\states/00/history.pickle
E:\scstGCN-main\data\states/00/history.png


E:\scstGCN-main\impute.py:147: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  x = torch.tensor(x, device=model.device)


All genes have been saved in E:\scstGCN-main\data\cnts-super/..
